In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
%matplotlib inline

In [6]:
df = pd.read_csv('train.csv').drop(columns=['Id'])
X_train, X_test, y_train, y_test = train_test_split(MinMaxScaler().fit_transform(df.iloc[:,1:]).reshape((-1, 28, 28, 1)), to_categorical(df.iloc[:,0], num_classes=5), test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [10]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28,28,1)),
    BatchNormalization(),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.25),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(5, activation='softmax')
])
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999), loss="categorical_crossentropy", metrics=["accuracy"])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [11]:
model.fit(X_train, y_train, epochs=40, batch_size=128)
loss, acc = model.evaluate(X_test, y_test)
print('Accuracy: %.3f' % acc)

Train on 48000 samples
Epoch 1/40
INFO:tensorflow:batch_all_reduce: 24 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 24 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
48000/48000 [==============================] - 42s 878us/sample - loss: 0.8456 - accuracy: 0.6759
Epoch 2/40
48000/48000 [==============================] - 12s 240us/sample - loss: 0.6095 - accuracy: 0.7552
Epoch 3/40
48000/48000 [==============================] - 11s 230us/sample - loss: 0.5439 - accuracy: 0.7822
Epoch 4/40
48000/48000 [==============================] - 10s 215us/sample - loss: 0.4942 - accuracy: 0.8028
Epoch 5/40
48000/48000 [==============================] - 10s 211us/sample - loss: 0.4595 - accuracy: 0.8204
Epoch 6/40
48000/48000 [==============================] - 10s 217us/sample - loss: 0.4374 - accuracy: 0.8287
Epoch 7/40
48000/48

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 8, zoom_range = 0.1, shear_range = 0.3, width_shift_range=0.08, height_shift_range=0.08, vertical_flip=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
history = model.fit(datagen.flow(X_train, y_train, batch_size = 128), epochs = 40, validation_data = (X_test, y_test), steps_per_epoch=X_train.shape[0] // 128, callbacks = [reduce_lr])
score = model.evaluate(X_test, y_test)

print('Loss: {:.4f}'.format(score[0]))
print('Accuracy: {:.4f}'.format(score[1]))

  ...
    to  
  ['...']
Train for 375 steps, validate on 12000 samples
Epoch 1/40
375/375 [==============================] - 23s 63ms/step - loss: 0.6649 - accuracy: 0.7512 - val_loss: 0.3261 - val_accuracy: 0.8732
Epoch 2/40
375/375 [==============================] - 21s 55ms/step - loss: 0.5207 - accuracy: 0.7897 - val_loss: 0.3525 - val_accuracy: 0.8547
Epoch 3/40
375/375 [==============================] - 22s 58ms/step - loss: 0.4928 - accuracy: 0.8031 - val_loss: 0.3338 - val_accuracy: 0.8637
Epoch 4/40
375/375 [==============================] - 20s 54ms/step - loss: 0.4783 - accuracy: 0.8084 - val_loss: 0.4013 - val_accuracy: 0.8298
Epoch 5/40
375/375 [==============================] - 21s 56ms/step - loss: 0.4631 - accuracy: 0.8143 - val_loss: 0.6324 - val_accuracy: 0.7389
Epoch 6/40
375/375 [==============================] - 21s 56ms/step - loss: 0.4547 - accuracy: 0.8170 - val_loss: 0.3197 - val_accuracy: 0.8668
Epoch 7/40
375/375 [==============================] - 21s 57ms/s

In [3]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28,28,1)),
    BatchNormalization(),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.5),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(5, activation='softmax')
])
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999), loss="categorical_crossentropy", metrics=["accuracy"])

In [5]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data = (X_test, y_test), callbacks = [reduce_lr])
loss, acc = model.evaluate(X_test, y_test)
print('Accuracy: %.3f' % acc)

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 8s 174us/sample - loss: 0.3455 - accuracy: 0.8625 - val_loss: 0.2962 - val_accuracy: 0.8742
Epoch 2/50
48000/48000 [==============================] - 8s 165us/sample - loss: 0.3384 - accuracy: 0.8649 - val_loss: 0.4136 - val_accuracy: 0.8307
Epoch 3/50
48000/48000 [==============================] - 8s 163us/sample - loss: 0.3269 - accuracy: 0.8704 - val_loss: 0.2907 - val_accuracy: 0.8819
Epoch 4/50
48000/48000 [==============================] - 8s 165us/sample - loss: 0.3165 - accuracy: 0.8746 - val_loss: 0.2683 - val_accuracy: 0.8883
Epoch 5/50
48000/48000 [==============================] - 8s 162us/sample - loss: 0.3184 - accuracy: 0.8736 - val_loss: 0.2603 - val_accuracy: 0.8951
Epoch 6/50
48000/48000 [==============================] - 8s 166us/sample - loss: 0.3105 - accuracy: 0.8750 - val_loss: 0.4409 - val_accuracy: 0.8234
Epoch 7/50
48000/48000 [==========================